# Day 7 solutions

My solutions to [Day 7](http://adventofcode.com/2017/day/7): Rescursive Circus. 

I found the second part of this problem to be rather tricky - I tried to represent the tree as a dictionary/map, but unfortuantely the way I'd built the tree from the input (children as keys, parents as values) didn't really work well with how I wanted to solve the second part (starting from the top of the tree and moving down to the leaves). This meant I had to invert the tree, hence the somewhat messy code that has resulted. However, I certainly lot about maps in C++ along the way!

In [1]:
#include <iostream>
#include <vector>
#include <string>
#include <sstream>
#include <iterator>
#include <regex>
#include <fstream>
#include <algorithm>
#include <map>
#include <set>

using namespace std;

int recurse(map <string, pair<string,int>> tower, map <string, pair<vector<string>, int>> tree, 
             string, bool terminate);

In [2]:
auto part1(string filename) {
    map <string, string> tower;
    ifstream filestream(filename);
    string s;  
    
    while (getline(filestream, s)) {
        vector <string> names;
        regex re("([a-z]{4,8})");
        copy(sregex_token_iterator(s.begin(), s.end(), re, 1), sregex_token_iterator(),
          back_inserter(names));
        tower.insert(pair<string,string>(*names.begin(),""));
        for (auto it=names.begin()+1; it!=names.end(); ++it) {
            tower[*it] = *names.begin();
        }
    }
    
    // search for key with "" as value
    for (auto it=tower.begin(); it!=tower.end(); ++it) {
        if (it->second == "") return it->first;
    }
    return string("None found");
}

In [3]:
part1("day7.txt")

(std::basic_string<char>) "ahnofa"


In [4]:
int recurse(map <string, pair<string,int>> tower, map <string, pair<vector<string>, int>> tree, 
             string node, bool terminate) {
    if (terminate) return 0;
    if (tree[node].second > 0) return tree[node].second;
    
    int weight = tower[node].second;
    if (tree[node].first.size() > 0) { // do the recursion
        map <int, string> map_weights; 
        for (auto it=tree[node].first.begin(); it!=tree[node].first.end(); ++it) {
            int w = recurse(tower, tree, *it, terminate);
            
            if (terminate) return 0;
            
            map_weights[w] = *it;
            weight+= w;
        }

        if (map_weights.size() > 1 && !(terminate)) { // if dictionary is larger than 1, have multiple weights
            terminate = true;
            // going assume the wrong one is always the higher
            auto last = map_weights.rbegin()->second;
            cout << "Current weight of " << last << " is " << tower[last].second << 
                ". New weight is " << tower[last].second - abs(map_weights.rbegin()->first - map_weights.begin()->first);
            cout << '\n';
            return 0;
        }
    } 
    tree[node].second = weight;
    return weight;
}

In [5]:
auto part2(string filename) {
    map <string, pair<string,int>> tower; // leaf, (parent, weight)
    ifstream filestream(filename);
    string s;  
    
    while (getline(filestream, s)) {
        vector <string> names;
        regex re("([a-z]{4,8})");
        copy(sregex_token_iterator(s.begin(), s.end(), re, 1), sregex_token_iterator(),
          back_inserter(names));
        regex num("\\d+");
        smatch ma;
        regex_search(s, ma, num);
        tower.insert(pair<string,pair<string,int>>(*names.begin(),pair<string, int>("", stoi(ma[0]))));
        tower[*names.begin()].second = stoi(ma[0]);
        for (auto it=names.begin()+1; it!=names.end(); ++it) {
            tower.insert(pair<string,pair<string,int>>(*it,pair<string, int>(*names.begin(), 0)));
            tower[*it].first = *names.begin();
        }
    }
    
    string top;
    
    for (auto it=tower.begin(); it!=tower.end(); ++it) {
        if ((it->second).first == "") {
            top = it->first;
            break;
        }
    }
    
    // invert previous tree
    map <string, pair<vector<string>, int>> tree; // parent, (list of children, sum of weights)
    vector <string> blank;
    for (auto it=tower.begin(); it!=tower.end(); ++it) {
        tree.insert(pair<string, pair<vector<string>, int>>((it->second).first, pair<vector<string>, int>(blank, 0)));
        tree[(it->second).first].first.push_back(it->first); 
    }
    
    recurse(tower, tree, top, false);
}

Note that the output for this is extraneous due to the recursion - only need to pay attention to first line.

In [6]:
part2("day7.txt")

Current weight of ltleg is 808. New weight is 802
Current weight of uppcjl is 9128. New weight is -4836
Current weight of xdpxpu is 65117. New weight is -4758
